In [ ]:
import os
import dotenv
import openai
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript

dotenv.load_dotenv()
# Replace with your OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI API client
openai.api_key = api_key

# Function to interact with ChatGPT
def chat_with_gpt(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=3500
    )
    return response.choices[0].text.strip()

# Create widgets
chat_output = widgets.HTML()
user_input = widgets.Textarea(
    placeholder="Type your message...",
    layout=widgets.Layout(width="80%"),
)
send_button = widgets.Button(
    description="Send",
    layout=widgets.Layout(width="10%", height="32px"),
)
clear_button = widgets.Button(
    description="Clear Conversation",
    layout=widgets.Layout(width="20%", height="32px"),
)

# Add icons to buttons
send_button.icon = "paper-plane"  # You can use other icons from FontAwesome or another source
clear_button.icon = "trash"       # You can use other icons from FontAwesome or another source

send_button.style.button_color = "#4CAF50"
send_button.style.font_weight = "bold"
clear_button.style.button_color = "#f44336"  # Red color for clearing

# Apply CSS styles for the send button
send_button.add_class("send-button")

# Style the chat output with a darker color scheme
chat_output.add_class("chat-output")

# Function to handle user input and display ChatGPT responses
def handle_send(sender):
    user_message = user_input.value.strip()
    if user_message.lower() == 'exit':
        clear_output()
        return
    if user_message:
        user_message_html = f'<strong class="user-message">You:</strong> {user_message}<br>'
        chat_output.value += user_message_html

        # Add a loading indicator while waiting for the response
        chat_output.value += '<div class="loading-indicator">AI is thinking...</div>'

        # Use a callback to fetch and display the AI response asynchronously
        def async_callback(*args):
            chat_response = chat_with_gpt("\n".join(conversation + [user_message]))
            chat_response_html = f'<strong class="ai-message">ChatGPT:</strong> {chat_response}<br>'
            chat_output.value += chat_response_html
            conversation.append(user_message)
            conversation.append(chat_response)

            # Remove the loading indicator
            chat_output.value = chat_output.value.replace('<div class="loading-indicator">AI is thinking...</div>', '')

        widgets.interact(async_callback)

    user_input.value = ""

# Function to handle clearing conversation
def handle_clear(sender):
    global conversation
    conversation = []
    chat_output.value = ""

clear_button.on_click(handle_clear)

# Bind the handle_send function to the button's click event
send_button.on_click(handle_send)

# JavaScript to enable sending by pressing Enter
enter_key_handler = Javascript("""
document.querySelector(".widget-textarea").addEventListener("keydown", function(e) {
    if (e.key === "Enter" && !e.shiftKey) {
        e.preventDefault();
        document.querySelector(".send-button").click();
    }
});
""")
display(enter_key_handler)

# Start a conversation
conversation = ["ChatGPT: Hello! How can I assist you today?"]

# Create a container for the input widgets
input_container = widgets.HBox([user_input, send_button, clear_button])

# Display widgets with enhanced styling and layout
display(widgets.VBox([chat_output, input_container], layout=widgets.Layout(padding="20px", width="80%")))

# Apply CSS styling for a visually appealing interface
style = HTML("""
<style>
body {
    background-color: #000000; /* Light background color for the entire interface */
    color: #333; /* Dark text color for the entire interface */
    font-family: Arial, sans-serif; /* Choose a readable font */
    margin: 0;
    padding: 0;
}

.chat-output {
    padding: 10px;
    background-color: #000000; 
    color: #FFFFFF;
    border-radius: 10px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); /* Subtle box shadow for chat history */
    overflow-y: auto;
    max-height: 300px;
}

.textarea-box {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-top: 20px;
    font-color: #FFFFFF
}

.textarea-box textarea {
    resize: none;
    border-radius: 5px;
    border: 1px solid #ccc;
    padding: 10px;
    font-size: 16px;
    font-color: #FFFFFF
    background-color: #404040; /* Lighter background color for textarea */
    color: #333; /* Dark text color for textarea */
    flex-grow: 1;
}

/* Style "You" text to differentiate user messages */
.user-message {
    color: #ADD8E6; /* Blue color for "You" text */
}

/* Style the AI's response */
.ai-message {
    color:#90EE90; /* Green color for AI responses */
}

/* Loading indicator style */
.loading-indicator {
    text-align: center;
    padding: 10px;
    font-style: italic;
    color: #888;
}
</style>
""")
display(style)
